<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd

In [2]:
VARIABLE_DICT_BIOAGEL = {
    'pregplan': 'PREGNANCY_PLANNED'
}

In [3]:
df = pd.read_stata('../data/bioagel.dta')

In [4]:
df

,hhnr,hhnrakt,persnr,persnre,syear,bioage,test,mode,sample1,paround,...,specned7_fid,specned8_fid,specned9_fid,stcaremn_fid,stcareyr_fid,timfamod_fid,timfamoh_fid,timfathd_fid,timfathh_fid,vistfrnd_fid
0,60,120308,1203003,609103,2014,[1] bioage01,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,...,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,-5,-5,-5,-5,-5,[-5] In Fragebogenversion nicht enthalten
1,868,79880,798803,798802,2006,[1] bioage01,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[1] A Deutscher HV (West),[-5] In Fragebogenversion nicht enthalten,...,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,-5,-5,-5,-5,-5,[-5] In Fragebogenversion nicht enthalten
2,868,79880,798803,798802,2009,[3] bioage03,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[1] A Deutscher HV (West),[-5] In Fragebogenversion nicht enthalten,...,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,-5,-5,-5,-5,-5,[-5] In Fragebogenversion nicht enthalten
3,868,79880,798803,798802,2012,[6] bioage06,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[1] A Deutscher HV (West),[-5] In Fragebogenversion nicht enthalten,...,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,-5,-5,-5,-5,-5,[-5] In Fragebogenversion nicht enthalten
4,868,79880,798803,798802,2014,[81] bioage08a,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,...,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,-5,-5,-5,-5,-5,[-5] In Fragebogenversion nicht enthalten
5,868,79880,798803,8603,2014,[82] bioage08b,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,...,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,-5,-5,-5,-5,-5,[-5] In Fragebogenversion nicht enthalten
6,868,79880,798804,798802,2012,[1] bioage01,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[1] A Deutscher HV (West),[-5] In Fragebogenversion nicht enthalten,...,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,-5,-5,-5,-5,-5,[-5] In Fragebogenversion nicht enthalten
7,868,79880,798804,798802,2014,[3] bioage03,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,...,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,-5,-5,-5,-5,-5,[-5] In Fragebogenversion nicht enthalten
8,868,127426,1274203,1274202,2008,[1] bioage01,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[1] A Deutscher HV (West),[-5] In Fragebogenversion nicht enthalten,...,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-5] In Fragebogenversion nicht enthalten,[-